# Animated Sessions
This code is meant to explain the animation of a session. 

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML
from matplotlib.patches import Ellipse, Circle
from matplotlib.collections import PatchCollection

idx = pd.IndexSlice
ROOT = os.environ['HOME'] + '/python/'


In [2]:
#load a dataset
sessions = pickle.load(open(ROOT + 'DATA_structures/session_dataframes/PSR_SESSIONS_DATAFRAME.p','rb'))
#get a single session
session = sessions.loc[idx['57','PL-saline',:,:],:]

In [4]:

def get_trial_info(sess, trial):
    if trial >= len(sess):
        return
    else:
        cardinality = sess['Choice'].iloc[trial]
        rewarded = sess['AR'].iloc[trial]
        GA = sess['GA'].iloc[trial]
        return cardinality, rewarded, GA

def create_canvas():
    fig, ax = plt.subplots(figsize=(10,6))
    ax.set_xlim(( 0, 2))
    ax.set_ylim((0, 2))

    # calculate asymmetry of x and y axes:
    x0, y0 = ax.transAxes.transform((0, 0)) # lower left in pixels
    x1, y1 = ax.transAxes.transform((2, 2)) # upper right in pixes
    dx = x1 - x0
    dy = y1 - y0
    maxd = max(dx, dy)
    width = .25 * maxd / dx
    height = .25 * maxd / dy

    #background color and goal labels
    ax.plot([1,1],[0,2], color='k')
    ax.axvspan(0, 1, facecolor='purple', alpha=0.2)
    ax.axvspan(1, 2, facecolor='green', alpha=0.2)
    ax.text(0.35, 1.8, 'WEST', fontsize=24)
    ax.text(1.4, 1.8, 'EAST', fontsize=24)
    ax.tick_params(axis='both', bottom=False, left=False)
    ax.set_xticklabels([])
    ax.set_yticklabels([])


    #these parameters will change every trial
    cardinality = 1

    #dynamic elements
    #trial information at bottom-left corner
    trial_text = ax.text(1.6, 0.1, 'Trial 1', fontsize=20)
    #current goal star
    goal = ax.scatter(0.7 + cardinality, 1.86, s=580, marker='*', color='y')
    #choice ellipse
    correct = ax.add_artist(Ellipse((0.5 + cardinality, rat_session_height),
                          width,
                          height,
                          facecolor='blue',
                          edgecolor='k',
                          linewidth=3,
                          alpha=0.5))


    #error triangle
    error = ax.add_patch(plt.Polygon([[0.4 + cardinality, rat_session_height + 0.2],
                                      [0.6 + cardinality, rat_session_height + 0.2],
                                      [0.5 + cardinality, rat_session_height - 0.2]],
                                      facecolor='red',
                                      edgecolor='k'))

    return fig, ax, trial_text, goal, correct, error


def init():
    #will include some polygons i suppose
    trial_text.set_text('')
    return trial_text,

def animate(i):
    trial_text.set_text('Trial %i' %i)
    cardinality, rewarded, GA = get_trial_info(session, i)
    nudgeX = (np.random.random() - 0.5)* 0.1
    nudgeY = (np.random.random() - 0.5)* 0.1
    if rewarded == 0:
        error.set_visible(True)
        error.set_xy([[0.4 + cardinality + nudgeX, rat_session_height + 0.2 + nudgeY],
                      [0.6 + cardinality + nudgeX, rat_session_height + 0.2 + nudgeY],
                      [0.5 + cardinality + nudgeX, rat_session_height - 0.2 + nudgeY]])
        correct.set_visible(False)
    else:
        correct.set_visible(True)
        correct.center = (0.5 + cardinality + nudgeX, rat_session_height + nudgeY)
        error.set_visible(False)

    goal.set_offsets([0.7 + GA, 1.86])

    return trial_text, correct, error, goal

    
#fixed height for model and rat choices
model_session_height = 0.6
rat_session_height = 1.4

noTrials = len(session)
fig, ax, trial_text, goal, correct, error = create_canvas()
anim = animation.FuncAnimation(fig, animate, frames=noTrials,
                              interval=500, blit=True, init_func=init)
# anim.save('animation.mp4', fps=2, 
#       extra_args=['-vcodec', 'h264', 
#                   '-pix_fmt', 'yuv420p'])

HTML(anim.to_html5_video())


## Defining a Class
Now that we have a working animation of a session, we create a class AnimatedSession that is initialized with a session and saves the video to a target folder. We now have to add also

In [8]:
import io
import base64
from Visualize import session_animator as SA

AnimatedSession = SA.AnimatedSession
session = sessions.loc[idx['57','IPSI-saline',:,:],:]
title = 'IPSI-Saline'
AS = AnimatedSession(session=session, title = title)
AS.save_to_file()
path_to_vid = '/Users/pablomartin/python/Results/session_animation/%s.mp4' %title
video = io.open(path_to_vid, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))